# Split Sample Analysis (Power users Vs. Casual users)

In [1]:
# Post Data
import pandas as pd
import numpy as np
import sqlite3
import pickle
import math
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT *
FROM posts
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
df = pd.read_sql_query(query, conn)
conn.close()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844597 entries, 0 to 5844596
Data columns (total 19 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   post_type             object 
 2   score                 int64  
 3   views                 int64  
 4   answers               int64  
 5   comments              int64  
 6   favorites             int64  
 7   creation_date         object 
 8   closed_date           object 
 9   accepted_answer_id    float64
 10  parent_id             float64
 11  owner_user_id         float64
 12  community_owned_date  object 
 13  tags                  object 
 14  title                 object 
 15  body                  object 
 16  last_editor_user_id   float64
 17  last_edit_date        object 
 18  last_activity_date    object 
dtypes: float64(4), int64(6), object(9)
memory usage: 847.2+ MB


In [11]:
# Preprocess
df = df[['id', 'post_type', 'creation_date', 'owner_user_id', 'answers', 'comments', 'parent_id']]
df = df.dropna(subset=['owner_user_id'])
df['owner_user_id'] = df['owner_user_id'].astype(int).astype(str)
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['year_month'] = df['creation_date'].dt.to_period('D')
df

,id,post_type,creation_date,owner_user_id,answers,comments,parent_id,year_month
0,69006420,answer,2021-09-01 00:00:18.070,7332046,0,1,69006229.0,2021-09-01
1,69006421,answer,2021-09-01 00:00:28.823,14471093,0,0,68746577.0,2021-09-01
2,69006422,answer,2021-09-01 00:00:35.537,523612,0,2,69006293.0,2021-09-01
3,69006423,question,2021-09-01 00:00:35.237,14087917,1,0,NaN,2021-09-01
4,69006426,question,2021-09-01 00:00:55.583,13091928,1,0,NaN,2021-09-01
...,...,...,...,...,...,...,...,...
5844592,77019853,answer,2023-08-31 23:56:47.647,5103949,0,0,76922631.0,2023-08-31
5844593,77019854,question,2023-08-31 23:57:28.633,2532775,0,2,NaN,2023-08-31
5844594,77019855,answer,2023-08-31 23:57:36.797,16844882,0,0,77019829.0,2023-08-31
5844595,77019856,question,2023-08-31 23:58:56.693,2604570,0,0,NaN,2023-08-31


In [12]:
df['post_type'].unique()

array(['answer', 'question', 'tag-wiki', 'tag-wiki-excerpt',
       'moderation-nomination'], dtype=object)

In [15]:
df_post = df[(df['post_type']=='question') | (df['post_type'] == 'answer')]
df_post

,id,post_type,creation_date,owner_user_id,answers,comments,parent_id,year_month
0,69006420,answer,2021-09-01 00:00:18.070,7332046,0,1,69006229.0,2021-09-01
1,69006421,answer,2021-09-01 00:00:28.823,14471093,0,0,68746577.0,2021-09-01
2,69006422,answer,2021-09-01 00:00:35.537,523612,0,2,69006293.0,2021-09-01
3,69006423,question,2021-09-01 00:00:35.237,14087917,1,0,NaN,2021-09-01
4,69006426,question,2021-09-01 00:00:55.583,13091928,1,0,NaN,2021-09-01
...,...,...,...,...,...,...,...,...
5844592,77019853,answer,2023-08-31 23:56:47.647,5103949,0,0,76922631.0,2023-08-31
5844593,77019854,question,2023-08-31 23:57:28.633,2532775,0,2,NaN,2023-08-31
5844594,77019855,answer,2023-08-31 23:57:36.797,16844882,0,0,77019829.0,2023-08-31
5844595,77019856,question,2023-08-31 23:58:56.693,2604570,0,0,NaN,2023-08-31


In [20]:
posts_count = df_post.groupby(['year_month', 'owner_user_id']).size().reset_index(name = 'Q_A_counts')
posts_count

,year_month,owner_user_id,Q_A_counts
0,2021-09-01,10000035,1
1,2021-09-01,10000229,1
2,2021-09-01,10002945,1
3,2021-09-01,10008173,4
4,2021-09-01,10008643,1
...,...,...,...
4658694,2023-08-31,9981953,1
4658695,2023-08-31,9985849,1
4658696,2023-08-31,9985972,1
4658697,2023-08-31,9990060,1


In [13]:
# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT *
FROM comments
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
comments_df = pd.read_sql_query(query, conn)
conn.close()

In [14]:
comments_df

,id,post_id,user_id,creation_date,score,text
0,121959106,69006188,8344820,2021-09-01 00:00:05.020,0,How large is arr?
1,121959108,63701554,1461061,2021-09-01 00:00:35.843,2,The backing fields for `init` properties are a...
2,121959109,69005989,4575398,2021-09-01 00:00:39.767,0,@P.... I just edited.
3,121959110,68994429,3287359,2021-09-01 00:00:40.690,0,"Great, that works! Thank you so much. Side not..."
4,121959111,69006395,14581079,2021-09-01 00:00:45.987,0,I in fact do have the permission! I'll check t...
...,...,...,...,...,...,...
8604543,135776595,77019842,478884,2023-08-31 23:59:21.113,0,`Workbooks.Open(file)` should give you a run-t...
8604544,135776596,77019484,19399290,2023-08-31 23:59:31.220,0,Could you share some of the data you are using...
8604545,135776597,77019774,12616482,2023-08-31 23:59:45.460,0,@Siguza thank you I will try there as well! Is...
8604546,135790379,74807261,8480069,2022-12-15 05:49:36.670,0,"**`Internet Issue ,Check your Internet Connect..."


In [22]:
# Preprocess
comments_df = comments_df.dropna(subset=['user_id'])
comments_df = comments_df.rename(columns = {'user_id':'owner_user_id'})
comments_df['owner_user_id'] = comments_df['owner_user_id'].astype(int).astype(str)
comments_df['creation_date'] = pd.to_datetime(comments_df['creation_date'])
comments_df['year_month'] = comments_df['creation_date'].dt.to_period('D')
comments_df

,id,post_id,owner_user_id,creation_date,score,text,year_month
0,121959106,69006188,8344820,2021-09-01 00:00:05.020,0,How large is arr?,2021-09-01
1,121959108,63701554,1461061,2021-09-01 00:00:35.843,2,The backing fields for `init` properties are a...,2021-09-01
2,121959109,69005989,4575398,2021-09-01 00:00:39.767,0,@P.... I just edited.,2021-09-01
3,121959110,68994429,3287359,2021-09-01 00:00:40.690,0,"Great, that works! Thank you so much. Side not...",2021-09-01
4,121959111,69006395,14581079,2021-09-01 00:00:45.987,0,I in fact do have the permission! I'll check t...,2021-09-01
...,...,...,...,...,...,...,...
8604543,135776595,77019842,478884,2023-08-31 23:59:21.113,0,`Workbooks.Open(file)` should give you a run-t...,2023-08-31
8604544,135776596,77019484,19399290,2023-08-31 23:59:31.220,0,Could you share some of the data you are using...,2023-08-31
8604545,135776597,77019774,12616482,2023-08-31 23:59:45.460,0,@Siguza thank you I will try there as well! Is...,2023-08-31
8604546,135790379,74807261,8480069,2022-12-15 05:49:36.670,0,"**`Internet Issue ,Check your Internet Connect...",2022-12-15


In [24]:
comments_count = comments_df.groupby(['year_month', 'owner_user_id']).size().reset_index(name = 'comments')
comments_count

,year_month,owner_user_id,comments
0,2021-09-01,-1,299
1,2021-09-01,10000857,1
2,2021-09-01,10001648,1
3,2021-09-01,10002945,1
4,2021-09-01,10007017,1
...,...,...,...
4104101,2023-08-31,9987870,1
4104102,2023-08-31,9990484,1
4104103,2023-08-31,999248,1
4104104,2023-08-31,9998081,1


In [33]:
df_merge = pd.merge(posts_count, comments_count, on = ['year_month', 'owner_user_id'], how = 'left')
df_merge

,year_month,owner_user_id,Q_A_counts,comments
0,2021-09-01,10000035,1,NaN
1,2021-09-01,10000229,1,NaN
2,2021-09-01,10002945,1,1.0
3,2021-09-01,10008173,4,10.0
4,2021-09-01,10008643,1,NaN
...,...,...,...,...
4658694,2023-08-31,9981953,1,NaN
4658695,2023-08-31,9985849,1,1.0
4658696,2023-08-31,9985972,1,NaN
4658697,2023-08-31,9990060,1,NaN


In [34]:
# Convert all null columns in comments variable to 0.
df_merge['comments'] = df_merge['comments'].fillna(0).astype(int)
df_merge['total_counts'] = df_merge['Q_A_counts']+df_merge['comments']
df_merge

,year_month,owner_user_id,Q_A_counts,comments,total_counts
0,2021-09-01,10000035,1,0,1
1,2021-09-01,10000229,1,0,1
2,2021-09-01,10002945,1,1,2
3,2021-09-01,10008173,4,10,14
4,2021-09-01,10008643,1,0,1
...,...,...,...,...,...
4658694,2023-08-31,9981953,1,0,1
4658695,2023-08-31,9985849,1,1,2
4658696,2023-08-31,9985972,1,0,1
4658697,2023-08-31,9990060,1,0,1


In [32]:
df_merge.isna().sum()

year_month       0
owner_user_id    0
Q_A_counts       0
comments         0
total_counts     0
dtype: int64

In [42]:
df_merge['total_counts'].quantile(0.85)

3.0

In [43]:
df_merge[df_merge['total_counts'] <= 3]

,year_month,owner_user_id,Q_A_counts,comments,total_counts
0,2021-09-01,10000035,1,0,1
1,2021-09-01,10000229,1,0,1
2,2021-09-01,10002945,1,1,2
4,2021-09-01,10008643,1,0,1
5,2021-09-01,10009777,1,0,1
...,...,...,...,...,...
4658694,2023-08-31,9981953,1,0,1
4658695,2023-08-31,9985849,1,1,2
4658696,2023-08-31,9985972,1,0,1
4658697,2023-08-31,9990060,1,0,1


In [44]:
df_merge[df_merge['total_counts'] > 3]

,year_month,owner_user_id,Q_A_counts,comments,total_counts
3,2021-09-01,10008173,4,10,14
9,2021-09-01,10016633,3,1,4
13,2021-09-01,10021876,1,6,7
24,2021-09-01,10035985,4,2,6
26,2021-09-01,10040513,1,5,6
...,...,...,...,...,...
4658671,2023-08-31,9894625,2,2,4
4658678,2023-08-31,9937026,2,3,5
4658687,2023-08-31,9971303,1,3,4
4658688,2023-08-31,9971404,1,3,4


In [49]:
avg_count = df_merge.groupby('owner_user_id')['total_counts'].mean().reset_index()
avg_count

,owner_user_id,total_counts
0,100,2.000000
1,10000015,1.000000
2,10000035,1.000000
3,10000042,1.000000
4,10000051,2.333333
...,...,...
1564962,9999918,1.000000
1564963,9999935,4.000000
1564964,9999957,1.000000
1564965,9999964,1.000000


In [122]:
avg_count['total_counts'].quantile(0.85)

2.0

In [123]:
avg_count['total_counts'].quantile(0.99)

5.5

In [124]:
avg_count['user_type'] = np.where(
    avg_count['total_counts'] < 2, 'casual',
    np.where(avg_count['total_counts'] < 5.5, 'intensive', 'top')
)

In [125]:
avg_count

,owner_user_id,total_counts,user_type
0,100,2.000000,intensive
1,10000015,1.000000,casual
2,10000035,1.000000,casual
3,10000042,1.000000,casual
4,10000051,2.333333,intensive
...,...,...,...
1564962,9999918,1.000000,casual
1564963,9999935,4.000000,intensive
1564964,9999957,1.000000,casual
1564965,9999964,1.000000,casual


In [128]:
avg_count.groupby('user_type').size().reset_index(name = "NumUsers")

,user_type,NumUsers
0,casual,1157244
1,intensive,391351
2,top,16372


In [126]:
avg_count.to_csv("split_power_casual.csv", index = False)